In [1]:
from ipywebrtc import CameraStream, ImageRecorder
import ipywidgets as widgets
from ipywidgets import VBox, HBox

import PIL.Image
import io

from fastai2.vision.all import *

In [2]:
#setup learner
inf = load_learner('/storage/lr_resnet18_1024_3epochs.pkl')

In [44]:
# setup gui elements

camera = CameraStream(constraints=
                      {'facing_mode': 'user',
                       'audio': False,
                       'video': { 'width': 640, 'height': 480 }
                       })

image_recorder = ImageRecorder(format='png', stream=camera)
image_recorder.recording=True

btn_classify = widgets.Button(description="Click Me!")
lbl_result = widgets.Output()

# callbacks
def classify(btn):
    global image_recorder
    
    im = PILImage.create(io.BytesIO(image_recorder.image.value))
    #im = PILImage.frombytes(image_recorder.image.value)
    lbl,_,_ = inf.predict(im)
    
    with lbl_result:
        print(lbl)
        
btn_classify.on_click(classify)

In [45]:
HBox([camera, VBox([image_recorder, HBox([btn_classify, lbl_result])])])